<a href="https://colab.research.google.com/github/visheshagrawal/IR2022_A1_96/blob/main/IR_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#Loading the colab essentials

## Mounting Drive

In [1]:
#Loading the colab essentials
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading the Target Directory

### Aditi Location

In [2]:
%cd /content/drive/MyDrive/IR/HW1/Dataset

/content/drive/.shortcut-targets-by-id/15TiqBLyk8RJBhmc1guzZPOx9pU0Ch0f6/IR/HW1/Dataset


##Installing some library


##Loading the Important Library

In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
# For missing values visualization
import missingno as msno
# For Plots
import matplotlib.pyplot as plt
import seaborn as sns
# For printing outputs in a beautiful way
from pprint import pprint
# For Normalization
from sklearn.preprocessing import MinMaxScaler
# For confusion matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score
# For cross validation
from sklearn.model_selection import cross_val_score
# For F_beta score
from sklearn.metrics import fbeta_score
# Classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
# For Model Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
# For splitting the Data into test and train
from sklearn.model_selection import train_test_split
# For F-beta
from sklearn.metrics import fbeta_score, make_scorer
# for legends
import matplotlib.patches as mpatches
#os annd list dir
import os
#json file processsing
import json
#NLP processing
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


# Download relevant data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## MatplotLib Settings

In [ ]:
%matplotlib inline
plt.style.use('seaborn')
sns.set(style="whitegrid", color_codes=True)

In [ ]:
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
#plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

#main functions

##traversing through files and saving the paths

In [4]:
files_list = os.listdir("Humor,Hist,Media,Food")
fpath=[]
count=0
for file in files_list:
    count += 1
    path2 = os.path.join("Humor,Hist,Media,Food", file)
    fpath.append(path2)

In [ ]:
json.dump(fpath, open("file_path_mapping1.json", "w"))


In [ ]:
count

1133

##preprocess

In [29]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer



In [33]:
def remove_special_char(text):
  punct_char = string.punctuation
  new_string=''
  for char in text:
    if char.isalnum() and not char.isdigit():
      if char not in punct_char:
        new_string+=char
  return new_string


In [ ]:

def preprocessing(text):
  stopWords = set(stopwords.words('english'))

  #Convert to lowercase
  # Stop words removed

  text = text.lower()
  #Tokenize
  text_tokens = word_tokenize(text)

  #stemming
  stemmer=PorterStemmer()
  #text_tokens = list([stemmer.stem(word) for word in text_tokens])
  #lemme
  lemma=WordNetLemmatizer()
  text_tokens = list([lemma.lemmatize(word) for word in text_tokens])
  tokens_wo_stopwords = []
  for token in text_tokens:
    if token not in stopWords:
      tokens_wo_stopwords.append(token)
  #removing punctuation marks
  tokens_wo_stopwords_punct = []
  for x in tokens_wo_stopwords:
    tokens_wo_stopwords_punct.append(remove_special_char(x))   
  #Removing blank space tokens
  validTokens = []
  for x in tokens_wo_stopwords_punct:
    if len(x)>1:
      validTokens.append(x)
  return validTokens

In [ ]:
def read_file(filePath):
    try:
        file = open(filePath, encoding="utf8")
        read = file.read().replace('\n', ' ')    
    except Exception as e:
        file = open(filePath, encoding="unicode_escape")
        read = file.read().replace('\n', ' ')
    file.close()
    return read

##inverted Index

#running the code


In [5]:
dataset_output={}

In [ ]:

def indexing(tokens,file_num):
  for i in tokens:
    if i  not in dataset_output:
      dataset_output[i]=[file_num]
    else:
      dataset_output[i].append(file_num)




In [ ]:

for i,filePath in enumerate(fpath):
  read=read_file(filePath)
  tokens=preprocessing(read) #tokenizing and preprocessing
  indexing(tokens,i) # performing indexin
  json.dump(dataset_output, open("output_lemma_final_1.json", "w"))

In [ ]:
len(dataset_output)

54855

In [ ]:
len(dataset_output)

54855

#Queries handling


##Preprocessing input

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [7]:
def pre_process_q(inp):
  stopWords = set(stopwords.words('english'))
  inp = inp.lower()
  text_tokens = word_tokenize(inp)
  #lemma  
  lemma=WordNetLemmatizer()
  text_tokens = list([lemma.lemmatize(word) for word in text_tokens])
  
  tokens_wo_stopwords = []
  for token in text_tokens:
    if token not in stopWords:
      tokens_wo_stopwords.append(token)
  #removing punctuation marks
  tokens_wo_stopwords_punct = []
  for x in tokens_wo_stopwords:
    tokens_wo_stopwords_punct.append(remove_special_char(x))   
  #Removing blank space tokens
  validTokens = []
  for x in tokens_wo_stopwords_punct:
    if len(x)>1:
      validTokens.append(x)
  return validTokens
  
  

In [8]:
def remove_special_char(text):
  punct_char = string.punctuation
  new_string=''
  for char in text:
    if char.isalnum() and not char.isdigit():
      if char not in punct_char:
        new_string+=char
  return new_string


##Query Processing

In [9]:
input_sen=""
ops=[]

In [10]:
def query_processing(input_sen,ops):
  tokens1=pre_process_q(input_sen,ops)

###x OR y

In [11]:
def x_or_y(term1_post,term2_post):
  term1_postings=set(term1_post)
  term2_postings=set(term2_post)
  or_=set.union(term1_post,term2_post)
  return or_


###x AND y

In [12]:
def x_and_y(term1_postings,term2_postings):
  term1_postings=set(term1_postings)
  term2_postings=set(term2_postings)
  and_=set.intersection(term1_postings,term2_postings)
  return and_

###x AND NOT y

In [13]:
def f1(t1,t2):
  candidate=[]
  for i in t1:
    if i not in t2:
      candidate.append(i)
  return candidate

###x OR NOT y


In [14]:
def f2(t1,t2):
  #or not
  candidate=[]
  for i in range(0,1133):
    if i not in t2:
      candidate.append(i)
  for j in t2:
    if j not in t1:
      candidate.append(i)
  return candidate

### Number of Comparisons

In [15]:
## merge algo
def num_com(term1_postings, term2_postings):
  i=0 #term1
  j=0 # term2
  result=0 # number of comparisions
  term1_postings=list(term1_postings)
  term2_postings=list(term2_postings)
  term1_postings.sort()
  term2_postings.sort()
  while(i< len(term1_postings) and j<len(term2_postings)):
    result+=1
    if (term1_postings[i]>term2_postings[j]):
      j+=1
    elif (term1_postings[i]<term2_postings[j]):
      i+=1
    else:
      j+=1
      i+=1
  return result

####Case 1 : AND

In [16]:
def num_com_and(term1_postings,term2_postings):
  a=num_com(term1_postings, term2_postings)
  return a


####Case 2 : OR

In [17]:
def num_com_or(term1_postings,term2_postings):
  a=num_com(term1_postings, term2_postings)
  return a

####Case 2 : AND NOT

In [18]:
def num_com_andNOT(term1_postings,term2_postings):
  universal_set=set(np.arange(1133))#edit number here
  set_diff=universal_set-set(term2_postings)
  a= num_com(term1_postings,set_diff)
  return a

####Case 1 : ORNOT

In [19]:
def num_com_orNOT(term1_postings,term2_postings):
  universal_set=set(np.arange(1133))#edit number here
  set_diff=universal_set-set(term2_postings)
  a= num_com(term1_postings,set_diff)
  return a


###getting a properly formatted string queryy

In [20]:
from tabulate import tabulate

def get_query(out_,inp,operands,pre_terms):
  #out_ is the indexing file
  #pre_terms is the pre processing term

  comparisons=0

  result=set(out_[pre_terms[0]])
  n=len(pre_terms)
  for i in range(1,n):
    op=operands[i-1]
    temp=out_[pre_terms[i]]
    if(op=='AND'):
      result=x_and_y(result,temp)
      comparisons+=num_com_and(result,temp)
    elif(op=='OR'):
      result=x_or_y(result,temp)
      comparisons+=num_com_or(result,temp)
    elif(op=='AND NOT'):
      result=f1(result,temp)#update
      comparisons+=num_com_andNOT(result,temp)
    elif(op=='OR NOT'):
      result=f2(result,temp)#up
      comparisons+=num_com_orNOT(result,temp)
    else:
      print("operand is invalid, please try again!")
      return -1
  print("docs matched",len(result))
  print("number of comparisons",comparisons)
  mapping=json.load(open('file_path_mapping1.json'))
  data= list(map(lambda i:(i,mapping[i]),result)) # list of tuples consisting of document id and their location
  data.sort()
  print(tabulate(data,headers=['Document ID','Location']))


In [21]:
from collections import defaultdict

def main_():
  n=int(input())
  for i in range(n):
    sentence=input("enter the sentence: ")
    operands_=list(map(str.strip,input("Enter the operands:- ").upper().split(",")))
    out_=json.load(open("output_lemma_final_1.json")) #out_ is the indexing file
    out_=defaultdict(lambda:[],out_)
    #print(len(out_['kid']))
    for item in out_:
      out_[item]=list(set(out_[item]))
    terms=pre_process_q(sentence)
    #print(len(out_['kid']))
    print("terms",terms)
    #print("out_",out_.keys())
    get_query(out_,sentence,operands_,terms)

In [22]:
main_()


1
enter the sentence: the lion stood proudly for a moment
Enter the operands:- OR, OR, OR
terms ['lion', 'stood', 'proudly', 'moment']
docs matched 218
number of comparisons 418
  Document ID  Location
-------------  ------------------------------------------
            8  Humor,Hist,Media,Food/conan.txt
           15  Humor,Hist,Media,Food/coyote.txt
           17  Humor,Hist,Media,Food/incarhel.hum
           22  Humor,Hist,Media,Food/insanity.hum
           25  Humor,Hist,Media,Food/cartoon_.txt
           29  Humor,Hist,Media,Food/ivan.hum
           43  Humor,Hist,Media,Food/lbinter.hum
           45  Humor,Hist,Media,Food/murphys.txt
           49  Humor,Hist,Media,Food/lif&love.hum
           50  Humor,Hist,Media,Food/lifeimag.hum
           52  Humor,Hist,Media,Food/llong.hum
           58  Humor,Hist,Media,Food/luggage.hum
           62  Humor,Hist,Media,Food/lozerzon.hum
           70  Humor,Hist,Media,Food/m0dzmen.hum
           71  Humor,Hist,Media,Food/maecenas.hum
      